## Import packages

In [ ]:
import ck.kernel as ck
import pandas as pd
import numpy as np
import matplotlib as matplotlib
import matplotlib.pyplot as plt
import json
import os

In [ ]:
print "Collective Knowledge: v%s" % ck.version({})['version_str']
print "pandas: v%s" % pd.version.version
print "NumPy: v%s" % np.version.version
print "Matplotlib: v%s" % matplotlib.__version__
print "JSON: v%s" % json.__version__

In [ ]:
%matplotlib inline

## Find results

In [ ]:
dataset = 'SGEMM_NT'
data_uoa =  dataset + '-explore-n-lws'
module_uoa = 'experiment'

In [ ]:
r=ck.access({'action':'list_points', 'module_uoa':module_uoa, 'data_uoa':data_uoa})
if r['return']>0:
  print ("Error: %s" % r['error'])
  exit(1)

## Show results

### Table

In [ ]:
data_list  = []
index_list = []

for point in r['points']:
    with open(os.path.join(r['path'], 'ckp-%s.flat.json' % point)) as point_file:
        point_data = json.load(point_file)
    # Data.    
    Gflops_per_s = point_data.get("##characteristics#run#run_time_state#EXECUTION#Gflops/s#all")
    #match = point_data.get("##characteristics#run#run_time_state#RESULTS#match#all")
    data_list.append(Gflops_per_s) # + match)
    # Index.
    cl_file = point_data.get("##characteristics#run#run_time_state#METADATA#file#all_unique")[0]
    lws_j = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_j#all_unique")[0]
    lws_i = point_data.get("##characteristics#run#run_time_state#EXECUTION#lws_i#all_unique")[0]
    matrix_order = point_data.get("##characteristics#run#run_time_state#CMD_LINE_ARGS#matrix_order#all_unique")[0]
    index_list.append((cl_file, lws_j * lws_i, lws_j, lws_i, matrix_order))

In [ ]:
num_repetitions = 4
#ci = pd.MultiIndex.from_arrays([range(num_repetitions)*2, ['Gflops/s']*num_repetitions + ['Match?']*num_repetitions])
ci = pd.MultiIndex.from_arrays([range(num_repetitions)])
mi = pd.MultiIndex.from_tuples(names=['OpenCL program', 'LWS_jxi', 'LWS_j', 'LWS_i', 'Matrix order'], tuples=index_list)
df = pd.DataFrame(data=data_list, index=mi, columns=ci)
df.index = df.index.droplevel('OpenCL program') # not interested in as it's fixed here
df['mean'] = df[range(num_repetitions)].mean(axis=1)
df['std']  = df[range(num_repetitions)].std(axis=1)
df.sortlevel('LWS_jxi')

In [ ]:
# import re
# # left paren, whitespace, number, whitespace, comma, whitespace, number, whitespace, right paren
# r = '\(\s*(?P<lws_j>\d+)\s*,\s*(?P<lws_i>\d+)\s*\)'
# f = df.index.to_series().values
# [ m.groups() for t in f for m in [re.match(r, t[0])] ]

### Plot

In [ ]:
ymax = np.int64(df['mean'].max() + df['std'].max())
plot = df['mean'] \
    .unstack(level='Matrix order') \
    .plot(yerr=df['std'].unstack(level='Matrix order'),
        title='Gflops/s vs Local work size',
        kind='bar', figsize=(12,10), colormap=matplotlib.cm.autumn,
        ylim=(0, ymax), yticks=range(ymax))

In [ ]:
# plot = mean \
#     .unstack(level='Local work size') \
#     .plot(yerr=std.unstack(level='Local work size'),
#         title='Gflops/s vs Matrix order',
#         kind='bar', figsize=(12,8), colormap=matplotlib.cm.autumn,
#         ylim=(0, ymax), yticks=range(ymax))

## Dump results for paper

### Table

In [ ]:
num_repetitions = 4
mi_tex = pd.MultiIndex.from_tuples(tuples=index_list)
df_tex = pd.DataFrame(data=data_list, index=mi_tex)
df_tex.index = df_tex.index.droplevel(0) # not interested in as it's fixed here
df_tex = df_tex.sortlevel(0) # 'LWS_jxi'
df_tex['mean'] = df_tex[range(num_repetitions)].mean(axis=1)
df_tex['std']  = df_tex[range(num_repetitions)].std(axis=1)
df_tex = df_tex.loc[[16,32,64]]
df_tex

In [ ]:
with open('%s_tmp.tex' % data_uoa, 'w') as tex_file:
     tex_file.write(df_tex.to_latex())

### Plot

In [ ]:
plot.get_figure().savefig('%s_tmp.pdf' % data_uoa)
# plot.get_figure().savefig('%s_tmp.png' % data_uoa)